# Working with AWS DynamoDB


## Purpose
This notebook is desinged for AWS developers to play around and explore different ways to interact with AWS DynamoDB.


# Setup

## Import Libraries
We import all the required Python libraries.

In [1]:
import boto3
from botocore.exceptions import ClientError
import json
import decimal

In [2]:
boto3.__version__

'1.22.9'

In [3]:
#Determine which credentials(profile) to use embeded in .aws/config file.
boto3 = boto3.session.Session(profile_name='AWS-DEMO')

## Load DynamoDB in us-west-2 region

In [4]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

## Create a table named Movies. 


In [5]:
# Create a table named Movies
table = dynamodb.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title', 
                'AttributeType':'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='Movies')

# Print out some data about the table.
print(table.item_count)

0


## List the tables


In [6]:
dynamodb.meta.client.list_tables()

{'TableNames': ['Movies', 'Notes'],
 'ResponseMetadata': {'RequestId': 'IKF91F79SKP16J91Q6F0TT4S67VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 20 Jul 2022 14:42:50 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '33',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'IKF91F79SKP16J91Q6F0TT4S67VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3013821024'},
  'RetryAttempts': 0}}

## Load a Sample Data

- Download a sample data file that contains information about a few thousand movies from the Internet Movie Database (IMDb). The movie data is in JSON format, as shown in the following example. For each movie, there is a year, a title, and a JSON map named info.


In [ ]:
#!wget https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/samples/moviedata.zip

- unzip the zip file

In [ ]:
#!unzip moviedata.zip

- Load the json file into the "Movies" table. There are thousands of movies in this file, this could take a while :)

In [7]:
table = dynamodb.Table('Movies')

with open("moviedata.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies[:20]: # Iterate over the first 20 movies. There are more than 4000 movies.
        year = int(movie['year'])
        title = movie['title']
        info = movie['info']

        print("Adding movie:", year, title)

        table.put_item(
           Item={
               'year': year,
               'title': title,
               'info': info,
            }
        )

Adding movie: 2013 Rush
Adding movie: 2013 Prisoners
Adding movie: 2013 The Hunger Games: Catching Fire
Adding movie: 2013 Thor: The Dark World
Adding movie: 2013 This Is the End
Adding movie: 2013 Insidious: Chapter 2
Adding movie: 2013 World War Z
Adding movie: 2014 X-Men: Days of Future Past
Adding movie: 2014 Transformers: Age of Extinction
Adding movie: 2013 Now You See Me
Adding movie: 2013 Gravity
Adding movie: 2013 We're the Millers
Adding movie: 2013 Riddick
Adding movie: 2013 The Family
Adding movie: 2013 Star Trek Into Darkness
Adding movie: 2013 After Earth
Adding movie: 2013 The Great Gatsby
Adding movie: 2014 Divergent
Adding movie: 2013 We Are What We Are
Adding movie: 2013 Iron Man 3


## Create a new item

In [8]:
# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

In [9]:
table = dynamodb.Table('Movies')


title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))


PutItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "B156KAR7R3I28VDPDNDSHVTF5JVV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "server": "Server",
            "date": "Wed, 20 Jul 2022 14:44:29 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "2",
            "connection": "keep-alive",
            "x-amzn-requestid": "B156KAR7R3I28VDPDNDSHVTF5JVV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "2745614147"
        },
        "RetryAttempts": 0
    }
}


## Read an Item

In [10]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

try:
    response = table.get_item(
        Key={
            'year': year,
            'title': title
        }
    )
except ClientError as err:
    print(err.response['Error']['Message'])
else:
    item = response['Item']
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))

GetItem succeeded:
{
    "year": 2015,
    "info": {
        "rating": 0,
        "plot": "Nothing happens at all."
    },
    "title": "The Big New Movie"
}


## Update an Item

In [11]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating=:r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

UpdateItem succeeded:
{
    "Attributes": {
        "info": {
            "actors": [
                "Larry",
                "Moe",
                "Curly"
            ],
            "plot": "Everything happens all at once.",
            "rating": 5.5
        }
    },
    "ResponseMetadata": {
        "RequestId": "DFJL1569CMQ12B4ISUVIQS4NO7VV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "server": "Server",
            "date": "Wed, 20 Jul 2022 14:45:09 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "156",
            "connection": "keep-alive",
            "x-amzn-requestid": "DFJL1569CMQ12B4ISUVIQS4NO7VV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "3767510606"
        },
        "RetryAttempts": 0
    }
}


## Conditional delete

In [12]:
table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

print("Attempting a conditional delete...")

try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.rating <= :val",
        ExpressionAttributeValues= {
            ":val": decimal.Decimal(6)
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print(e.response['Error']['Message'])
    else:
        raise
else:
    print("DeleteItem succeeded:")
    print(json.dumps(response, indent=4, cls=DecimalEncoder))

Attempting a conditional delete...
DeleteItem succeeded:
{
    "ResponseMetadata": {
        "RequestId": "ANE1PQPM87IJ48GHES3JSIN937VV4KQNSO5AEMVJF66Q9ASUAAJG",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "server": "Server",
            "date": "Wed, 20 Jul 2022 14:46:12 GMT",
            "content-type": "application/x-amz-json-1.0",
            "content-length": "2",
            "connection": "keep-alive",
            "x-amzn-requestid": "ANE1PQPM87IJ48GHES3JSIN937VV4KQNSO5AEMVJF66Q9ASUAAJG",
            "x-amz-crc32": "2745614147"
        },
        "RetryAttempts": 0
    }
}


## Query - All Movies Released in a Year

In [13]:
from boto3.dynamodb.conditions import Key, Attr

table = dynamodb.Table('Movies')

print("Movies from 2013")

response = table.query(
    KeyConditionExpression=Key('year').eq(2013)
)

for i in response['Items']:
    print(i['year'], ":", i['title'])

Movies from 2013
2013 : After Earth
2013 : Gravity
2013 : Insidious: Chapter 2
2013 : Iron Man 3
2013 : Now You See Me
2013 : Prisoners
2013 : Riddick
2013 : Rush
2013 : Star Trek Into Darkness
2013 : The Family
2013 : The Great Gatsby
2013 : The Hunger Games: Catching Fire
2013 : This Is the End
2013 : Thor: The Dark World
2013 : We Are What We Are
2013 : We're the Millers
2013 : World War Z


## Query - All Movies Released in a Year with Certain Titles

In [14]:
table = dynamodb.Table('Movies')

print("Movies from 2013 - titles A-P, with genres and lead actor")

response = table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={ "#yr": "year" }, # Expression Attribute Names for Projection Expression only.
    KeyConditionExpression=Key('year').eq(2013) & Key('title').between('A', 'P')
)

for i in response[u'Items']:
    print(json.dumps(i, indent=4, cls=DecimalEncoder))

Movies from 2013 - titles A-P, with genres and lead actor
{
    "info": {
        "actors": [
            "Jaden Smith"
        ],
        "genres": [
            "Action",
            "Adventure",
            "Sci-Fi"
        ]
    },
    "year": 2013,
    "title": "After Earth"
}
{
    "info": {
        "actors": [
            "Sandra Bullock"
        ],
        "genres": [
            "Drama",
            "Sci-Fi",
            "Thriller"
        ]
    },
    "year": 2013,
    "title": "Gravity"
}
{
    "info": {
        "actors": [
            "Patrick Wilson"
        ],
        "genres": [
            "Horror",
            "Thriller"
        ]
    },
    "year": 2013,
    "title": "Insidious: Chapter 2"
}
{
    "info": {
        "actors": [
            "Robert Downey Jr."
        ],
        "genres": [
            "Action",
            "Adventure",
            "Sci-Fi"
        ]
    },
    "year": 2013,
    "title": "Iron Man 3"
}
{
    "info": {
        "actors": [
            "Je

## Scan

The following program scans the entire Movies table, which contains approximately 5,000 items. The scan specifies the optional filter to retrieve only the movies from the 2010s and discard all the others.

In [15]:
table = dynamodb.Table('Movies')

fe = Key('year').between(2010, 2020)
pe = "#yr, title, info.rating"
# Expression Attribute Names for Projection Expression only.
ean = { "#yr": "year", }
esk = None


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
    )

for i in response['Items']:
    print(json.dumps(i, cls=DecimalEncoder))

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExpressionAttributeNames= ean,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )

    for i in response['Items']:
        print(json.dumps(i, cls=DecimalEncoder))

{"info": {"rating": 4.9}, "year": 2013, "title": "After Earth"}
{"info": {"rating": 8.2}, "year": 2013, "title": "Gravity"}
{"info": {"rating": 7.1}, "year": 2013, "title": "Insidious: Chapter 2"}
{"info": {"rating": 7.4}, "year": 2013, "title": "Iron Man 3"}
{"info": {"rating": 7.3}, "year": 2013, "title": "Now You See Me"}
{"info": {"rating": 8.2}, "year": 2013, "title": "Prisoners"}
{"info": {"rating": 6.8}, "year": 2013, "title": "Riddick"}
{"info": {"rating": 8.3}, "year": 2013, "title": "Rush"}
{"info": {"rating": 7.9}, "year": 2013, "title": "Star Trek Into Darkness"}
{"info": {"rating": 6.5}, "year": 2013, "title": "The Family"}
{"info": {"rating": 7.3}, "year": 2013, "title": "The Great Gatsby"}
{"year": 2013, "title": "The Hunger Games: Catching Fire"}
{"info": {"rating": 7.2}, "year": 2013, "title": "This Is the End"}
{"year": 2013, "title": "Thor: The Dark World"}
{"info": {"rating": 6.2}, "year": 2013, "title": "We Are What We Are"}
{"info": {"rating": 7.2}, "year": 2013, 

## DELETE THE TABLE

In [16]:
table = dynamodb.Table('Movies')

table.delete()

{'TableDescription': {'TableName': 'Movies',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-west-2:465968594676:table/Movies',
  'TableId': '19bcd598-fc47-4284-af2f-8f7b261a74d6'},
 'ResponseMetadata': {'RequestId': '1F117J1TJE12NCSFHF86DVSG1BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 20 Jul 2022 14:48:12 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '316',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1F117J1TJE12NCSFHF86DVSG1BVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '52347199'},
  'RetryAttempts': 0}}